<a href="https://colab.research.google.com/github/rogerioag/rea-comp04-compiladores/blob/main/jupyter-notebooks/03-comp-analise-semantica-cmmsemantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise Semântica

![](data:image/png;base64,)


A Análise Semântica consiste em uma "segunda passada" do compilador no código, pois o código fonte é novamente analisado desde o início na **AST**, porém agora para realizar a anotação de atributos, inferência de tipos e verificações se as construções sintáticas fazem algum sentido para a linguagem.

Como entrada para a _Análise Semântica_ temos a Árvore Sintática Abstrata (AST) produzida pela fase anterior, a _Análise Sintática_ com a utilização da lista de _tokens_ identificados pela _Análises Léxica_.

Nesta terceira parte do desenvolvimento do projeto do Compilador, a **AST** deve ser percorrida para a realização da análise sensível ao contexto e a geração ou finalização da _Tabela de Símbolos_. 

O principal motivo da **Análise Semântica** é verificar se existem erros de contexto para a linguagem **C-** apresentada na primeira e segunda parte deste tutorial. Um conjunto de Regras Semânticas devem ser verificadas.

## Regras Semânticas

As **Regras Semânticas** que precisam ser verificadas para a identicação de erros semânticos ou de contexto são:

1. **Funções e Procedimentos.** Cada um dos _identificadores de função_, nome e quantidade de parâmetros formais deve ser verificado, os parâmetros formais devem estar vinculados a identificadores de variáveis locais.
 
2. **Função Principal.** Todo programa escrito em **C-** deve ter uma *função principal* declarada, com a identificação `main`. Verificar a existência de uma função principal que inicializa a execução do código. Caso contrário, deve apresentar a mensagem:
   
	**Erro:** *Função principal não declarada.*

	A função principal normalmente é do tipo `int`, mas tambéém é aceito o retorno do tipo `void`, assim é esperado que seu retorno seja um valor inteiro ou vazio, do contrário a mensagem deve ser emitida:
	
	**Erro:** Função principal deveria retornar valor inteiro ou vazio.

2. **Funções e Procedimentos.** A quantidade de parâmetros reais de uma chamada de função/procedimento `func` deve ser igual a quantidade de parâmetros formais da sua definição. Caso contrário, gerar a mensagem:
	
	**Erro:** Chamada à função 'func' com número de parâmetros menor que o declarado.

3. **Funções e Procedimentos.** Uma função deve retornar um valor de tipo compatível com o tipo de retorno declarado. Se a função `main` que é declarada com retorno `int`, não apresenta um `return(0)`, a mensagem deve ser gerada:

	**Erro:** Função 'main' deveria retornar inteiro, mas retorna vazio.

	Funções precisam ser declaradas antes de serem chamadas/ativadas. Caso contrário a mensagem de erro deve ser emitida: 
   
	**Erro:** Chamada a função 'func' que não foi declarada.

	Uma função qualquer não pode fazer uma chamada à função `main`. Devemos verificar ser existem alguma chamada para a função `main` partindo de qualquer outra função do programa.
   
	**Erro:** Chamada para a função principal não permitida.

	Uma função pode ser declarada e não utilizada. Se isto acontecer uma aviso deverá ser emitido:
	
	**Aviso:** Função 'func' declarada, mas não utilizada.

	Se a função `main` fizer uma chamada para ela mesmo, a mensagem de aviso deve ser emitida:
	
	**Aviso:** Chamada recursiva para 'main'.

4. __Variáveis.__ O __identificador de variáveis locais e globais__: nome, tipo e escopo devem ser aramazenados na Tabela de Símbolos.
   Variáveis devem ser **declaradas**, **inicializadas** antes de serem **utilizadas** (leitura).
   
   Lembrando que uma variável pode ser declarada:
    - no escopo do procedimento (como expressão ou como parâmetro formal);
    - no escopo global

  *Warnings*/Avisos deverão ser mostrados quando uma variável for declarada mas nunca utilizada. Se uma variável 'a' for apenas declarada e não for inicializada (escrita) ou não for utilizada (não lida), o analisador deve gerar a mensagem:
	
  **Aviso:** Variável 'a' declarada e não utilizada.
	
  Se houver a tentativa de leitura ou escrita de qualquer variável não declarada a seguinte mensagem:
	
	**Erro:** Variável 'a' não declarada.
	
  Se houver a tentativa de leitura de uma variável 'a' declarada, mas não inicializada:
	
	**Aviso:** Variável 'a' declarada e não inicializada.
	
  *Warnings* deverão ser mostrados quando uma variável for declarada mais de uma vez. Se uma variável 'a' for declarada duas vezes no mesmo escopo, o aviso deve ser emitido:
	
	**Aviso:** Variável 'a' já declarada anteriormente

5. **Atribuição.** Na atribuição devem ser verificados se os tipos são compatíveis. Por exemplo, uma variável `a` recebe uma expressão `b + c`. Os tipos declarados para `a` e o tipo resultado da inferência do tipo da expressão `b + c` deverão ser compatíveis. Se `b` for `int` e `c` for `int`, o tipo resultante da expressão será também `int`.
  Se assumirmos, por exemplo, que `b` é do tipo `int` e `c` do tipo `float`, o resultado pode ser `float` (se assumirmos isso para nossa linguagem). O que faria a atribuição `a := b + c` apresentar tipos diferentes e a mensagem deve ser apresentada:
	
	**Aviso:** Atribuição de tipos distintos: 'a' int e 'expressão' float

  O mesmo pode acontecer com a atribuição de um retorno de uma função, se os tipos forem incompatíveis o usuário deve ser avisado:
	
	**Aviso:** Atribuição de tipos distintos 'a' float e 'func' retorna int

6. **Coerções implícitas.** *Warnings* deverão ser mostrados quando ocorrer uma coerção implícita de tipos (int<->float). Atribuição de variáveis ou resultados de expressões de tipos distintos devem gerar a mensagem:
	
	**Aviso:** Coerção implícita do valor de 'x'.

	**Aviso:** Coerção implícita do valor retornado por 'func'.

7. **Arranjos.** Na linguagem `C-` é possível declarar arranjos, pela sintaxe da linguagem o índice de um arranjo é inteiro e isso deve ser verificado. Na tabela de símbolos devemos armazenar se uma variável declarada tem um tipo, se é uma variável escalar ou um vetor ou uma matriz. Podemos armazenar um campo 'dimensões', que '0': escalar, '1': arranjo unidimensional (vetor) e '2': arranjo bidimensional (matriz) e assim por diante.

Encontrado a referência a um arranjo, seu o índice, seja um número, variável ou expressão deve ser um `inteiro`. Do contrário, a mensagem deve ser gerada: 
	
  __Erro:__ Índice de array 'X' não inteiro.

Se o acesso ao elemento do arranjo estiver fora de sua definição, por exemplo um vetor `A` é declarado como tendo `10` elementos (0 a 9) e há um acesso ao `A[10]`, a mensagem de erro deve ser apresentada: 
	
  **Erro:** índice de array 'A' fora do intervalo (out of range)

8. E outros situações descritas nos arquivos de testes.

Para a realização da análise dos erros apontados, a construção da **Tabela de Símbolos** deve ser realizada adequadamente. Uma Tabela de Símbolos pode começar a ser construída desde as análises anteriores. Por exemplo, na **Análise Léxica**, onde temos os `TOKENS` e os lexemas que casaram com a expressão regular que gerou cada um desses tokens. Temos também o número da linha e da coluna que o lexema foi encontrado no código fonte de entrada. Desta forma, as entradas da Tabela de Símbolos podem ir sendo preenchidas com essas informações iniciais <TOKEN, LEXEMA, LINHA, COLUNA>.

Na **Análise Sintática** quando o reconhecimento entrar na função que trata a regra, por exemplo, na regra de _declaração de variáveis_, todas as variáveis da lista de identificadores irão assumir o tipo declarado:

``int a, b, c[10]``

Que entraria na regra:

``var-declaration ::= type-specifier ID ; | type-specifier ID [ NUM ] ; | type-specifier ID [ NUM ] [ NUM ] ;``

A declaração indica que os elementos da `lista_variaveis`, que são as variáveis escalares `a` e `b` e o arranjo unidimensional `c` terão por declaração o `tipo` que no exemplo é `inteiro`.

## Exemplo

```.pascal
inteiro: a, b[10], c[3][5]

b := 10

inteiro func(inteiro: x, inteiro: y)
  inteiro: res
  res = x + y;
  retorna(res) 
fim

inteiro principal()
  leia(a)
  leia(b)

  b[0] := a
  b[1] := b
  c[0][1] := func(a,b)

  escreva(c[3])
  retorna(0)
fim
```

### Tabela de Símbolos

| Token  | Lexema  | Tipo  |	dim  | tam_dim1 | tam_dim2  | escopo  | init  | linha  |  coluna  |
| :----: | :-----: | :----: | :-----| :--- | :------ | :---- | :----- | :------- |
|ID|"a"|int|0|1|0|global|N|1|9|
|ID|"b"|int|1|10|0|global|N|1|9|
|ID|"c"|int|2|3|5|global|N|1|9|


## Árvore Sintática Abstrata

Após a análise semântica e geração de dados referentes a esta passagem, é esperado como saída uma árvore sintática abstrata anotada (ASTO).

Tomemos como exemplo uma atribuição ``a := b + c``, de acordo com a sintaxe da linguagem **TPP** temos a subárvore representada na Figura \ref{fig:atribuicao}.

![Atribuição expandida \label{fig:atribuicao}](https://github.com/rogerioag/rea-comp04-compiladores/blob/main/jupyter-notebooks/figuras/figura-atribuica.png?raw=1){#id .class width=95% height=95%}

Após a verificação das regras semânticas, pode-se fazer uma poda dos nós internos da árvore para facilitar a geração de código. A Figura \ref{fig:atribuicao:poda} apresenta a Árvore Sintática Abstrata simplificada.

![Atribuição depois da poda \label{fig:atribuicao:poda}](https://github.com/rogerioag/rea-comp04-compiladores/blob/main/jupyter-notebooks/figuras/figura-atribuica-asa.png?raw=1){#id .class width=35% height=35%}

O Código que deve ser gerado para uma ``atribuicao`` é apresentado no Código:

```.llvm
  %2 = load i32, i32* @b, align 4
  %3 = load i32, i32* @c, align 4
  %4 = add nsw i32 %2, %3
  store i32 %4, i32* @a, align 4
```

## Testes

Alguns casos de testes estão disponíveis no moodle institucional junto com essa especificação. Serão executados esses testes e outros testes que o professor julgar necessário durante a avaliação desta parte do trabalho.

## Documentação

Durante toda a disciplina o aluno criará uma documentação formal da implementação do compilador para a linguagem. Sendo os relatório com conteúdo acumulativo, isto é, as fases subsequentes irão complementar o conteúdo existente das fases anteriores. Neste ponto do trabalho o aluno deverá incluir na documentação:

- Estratégias utilizadas para a realização da análise sensível ao contexto;
- Estratégia utilizada para a geração da tabela de símbolos;
- Qualquer decisão de projeto referente à semântica da linguagem T++;

Utilize o formato de artigo da SBC [^1] para fazer o relatório.

[^1]: [Formato para publicação de artigos da SBC: http://www.sbc.org.br/documentos-da-sbc/summary/169-templates-para-artigos-e-capitulos-de-livros](http://www.sbc.org.br/documentos-da-sbc/summary/169-templates-para-artigos-e-capitulos-de-livros/878-modelosparapublicaodeartigos)

## Avaliação

Será avaliado se a análise sensível ao contexto executa a saída proposta (ASTO) e se todos os erros e *warnings* semânticos são mostrados corretamente para um exemplo simples escrito pelo professor/aluno. O essencial da linguagem deve ser mantido (conforme especificado no primeiro trabalho) para que o mesmo exemplo funcione para testar todos os trabalhos com o mínimo de alteração.

* Serão avaliados, dentre outros critérios:
	a) Da implementação:
	  + O funcionamento do programa.
	  + O capricho e a organização na elaboração do projeto.
	  + A corretude da implementação em relação ao que foi pedido no trabalho.
	  + A colocação em prática dos conceitos que foram discutidos em sala de aula de forma correta.
	  + A qualidade do projeto e da implementação (descrição e elaboração do projeto e o passo a passo da implementação).

 	b) Do relatório:
 	  + O conteúdo e a forma que foi apresentado, se o formato é o mesmo solicitado.
 	  + Organização das ideias e do processo de tradução.
 	  + O capricho na elaboração e na formatação do texto, bem como o conteúdo do texto.

* Não serão avaliados os trabalhos:
	a) Que chegarem fora do prazo.
 	b) Que não forem feitos nas ferramentas solicitadas.
 	c) Que não estão no formato especificado.
 	d) Que não foram compactados em um só arquivo.
 	e) Que não tiverem identificação (nome e matrícula).
 	f) Que forem cópias de outros trabalhos ou materiais da internet.
 	g) Que não seguirem todas estas instruções.

* Não se esqueça que o trabalho vale **10,0**  e contribui para o cálculo da nota final.

## Entrega e apresentação

O trabalho será **individual** e deverá ser entregue até o dia **03/08/2021** no moodle da disciplina em um pacote compactado e apresentado no mesmo dia. A estrutura do projeto com os arquivos do projeto (fonte e relatório) deve ser compactada (zipados) e o arquivo compactado deve ser enviado pelo moodle utilizando a opção de submissão "Trabalho 3a. parte - Análise Semântica", o nome do arquivo compactado deve seguir o padrão de nomes do formato.

Deverá ser especificado na entrega o mecanismo de execução da varredura para a realização da correção.

**Obs.:** Favor utilizar ZIP como forma de compactação. O Relatório deve ser entregue impresso, no horário da aula, para o professor.

## Referências